# Importar Bibliotecas Necessárias
Importe as bibliotecas necessárias, incluindo PySpark.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
#import pandas_profiling as pp

# Configurar Sessão Spark
Configure a sessão Spark para iniciar o contexto Spark.

In [3]:
# Configurar a sessão Spark com as configurações desejadas
# spark = SparkSession.builder \
#     .appName("Target Audience Study") \
#     .config("spark.sql.shuffle.partitions", "50") \
#     .config("spark.executor.memory", "2g") \
#     .config("spark.driver.memory", "2g") \
#     .getOrCreate()
    
    

#Melhor configuracao para o Spark no meu ambiente local

spark = SparkSession.builder \
    .appName("HackathonDM") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.5") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.sql.autoBroadcastJoinThreshold", "20MB") \
    .getOrCreate()

24/11/25 15:24:44 WARN Utils: Your hostname, MacBook-Pro-de-Thales.local resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
24/11/25 15:24:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 15:24:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Carregar Dados em um DataFrame Spark
Carregue os dados em um DataFrame Spark a partir de uma fonte de dados, como um arquivo CSV ou um banco de dados.

In [4]:
# Especificar o diretório contendo os arquivos .parquet
# fato_total_risco_trigg = "/Users/zaninth/Desktop/hackathon_pod_dm/fato_total_risco_trigg"
fato_total_risco_dm = "/Users/zaninth/Desktop/hackathon_pod/fato_total_risco_dm"

# Ler todos os arquivos .parquet no diretório em um único DataFrame
# df_trigg = spark.read.parquet(fato_total_risco_trigg)
df_dm = spark.read.parquet(fato_total_risco_dm)

# Mostrar o esquema do DataFrame resultante
# df_trigg.printSchema()
df_dm.printSchema()

# Obter o número de linhas
# num_rows_df_trigg = df_trigg.count()
num_rows_df_dm = df_dm.count()

# Obter o número de colunas
# num_cols_df_trigg = len(df_trigg.columns)
num_cols_df_dm = len(df_dm.columns)

# Mostrar o número de linhas e colunas
# print(f"O DataFrame df_trigg tem {num_rows_df_trigg} linhas e {num_cols_df_trigg} colunas.")
print(f"O DataFrame df_dm tem {num_rows_df_dm} linhas e {num_cols_df_dm} colunas.")

root
 |-- ch_cpf_hash: string (nullable = true)
 |-- dt_min_adesao: date (nullable = true)
 |-- dt_max_adesao: date (nullable = true)
 |-- vl_qnt_cartao: integer (nullable = true)
 |-- vl_min_score_colletion: integer (nullable = true)
 |-- vl_max_score_colletion: integer (nullable = true)
 |-- vl_min_dias_atraso: integer (nullable = true)
 |-- vl_max_dias_atraso: integer (nullable = true)
 |-- vl_min_saldo_devedor: decimal(26,6) (nullable = true)
 |-- vl_max_saldo_devedor: decimal(26,6) (nullable = true)
 |-- vl_total_saldo_devedor: decimal(26,6) (nullable = true)
 |-- vl_min_limite: decimal(15,2) (nullable = true)
 |-- vl_max_limite: decimal(15,2) (nullable = true)
 |-- vl_total_limite: decimal(15,2) (nullable = true)
 |-- ds_behaviour: string (nullable = true)
 |-- fl_cpf_ativo: string (nullable = true)
 |-- dt_ultima_compra: date (nullable = true)
 |-- fl_tem_dm_app: string (nullable = true)
 |-- dt_cadastro_dmapp: date (nullable = true)
 |-- dt_data_ultimo_acesso_app: date (nullabl

O DataFrame df_dm tem 173015422 linhas e 115 colunas.


# Estatísticas Descritivas

In [5]:
# Estatísticas descritivas para colunas numéricas
# df_dm.describe().show

In [6]:
# Listar tipos de dados de cada coluna
for coluna, tipo in df_dm.dtypes:
    print(f"{coluna}: {tipo}")


ch_cpf_hash: string
dt_min_adesao: date
dt_max_adesao: date
vl_qnt_cartao: int
vl_min_score_colletion: int
vl_max_score_colletion: int
vl_min_dias_atraso: int
vl_max_dias_atraso: int
vl_min_saldo_devedor: decimal(26,6)
vl_max_saldo_devedor: decimal(26,6)
vl_total_saldo_devedor: decimal(26,6)
vl_min_limite: decimal(15,2)
vl_max_limite: decimal(15,2)
vl_total_limite: decimal(15,2)
ds_behaviour: string
fl_cpf_ativo: string
dt_ultima_compra: date
fl_tem_dm_app: string
dt_cadastro_dmapp: date
dt_data_ultimo_acesso_app: date
fl_conta_avita: string
dt_ultimo_movimento: date
id_tipo_ultimo_movimento: string
vl_valor_ultimo_movimento: decimal(15,2)
ds_categoria_ultimo_movimento: string
dt_data_saldo: date
vl_saldo_dm_conta: decimal(15,2)
fl_tem_dmcred: string
id_cliente_nivel: int
dt_data_contratacao: date
dt_data_efetivacao: date
id_ciclo: int
vl_dias_atraso_dmcred: int
vl_saldo_devedor_dmcred: decimal(15,2)
fl_tem_dmpag: string
dt_cadastro_dmpag: date
fl_dmpag_ativo: string
dt_ultima_compra_d

## 6. Verificar Valores Nulos em Cada Coluna

In [14]:
from pyspark.sql.functions import col, sum, when
from pyspark.sql import Row

# # Contar valores nulos em cada coluna
# valores_nulos = df_dm.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_dm.columns])
# valores_nulos.show()
# Contar valores nulos em cada coluna e coletar os resultados


from pyspark.sql.functions import col, sum, when

# Contar o número total de linhas no DataFrame
total_linhas = df_dm.count()

# Contar valores nulos em cada coluna e coletar os resultados
valores_nulos = df_dm.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_dm.columns
])

# Coletar os resultados como um dicionário
nulos_dict = valores_nulos.collect()[0].asDict()

# Converter o dicionário em uma lista de Rows, adicionando o percentual de valores nulos
nulos_lista = [
    Row(
        Coluna=k, 
        Valores_Nulos=v, 
        Porcentagem_Nulos=round((v / total_linhas) * 100, 2)
    ) for k, v in nulos_dict.items()
]

# Criar um DataFrame Spark a partir da lista
valores_nulos_df = spark.createDataFrame(nulos_lista)

# Exibir os valores nulos e percentuais por coluna
valores_nulos_df.show(valores_nulos_df.count(), truncate=False)


+---------------------------------+-------------+-----------------+
|Coluna                           |Valores_Nulos|Porcentagem_Nulos|
+---------------------------------+-------------+-----------------+
|ch_cpf_hash                      |34           |0.0              |
|dt_min_adesao                    |3102173      |1.79             |
|dt_max_adesao                    |3102173      |1.79             |
|vl_qnt_cartao                    |0            |0.0              |
|vl_min_score_colletion           |67261947     |38.88            |
|vl_max_score_colletion           |67261947     |38.88            |
|vl_min_dias_atraso               |1579553      |0.91             |
|vl_max_dias_atraso               |1579553      |0.91             |
|vl_min_saldo_devedor             |2047754      |1.18             |
|vl_max_saldo_devedor             |2047754      |1.18             |
|vl_total_saldo_devedor           |2047754      |1.18             |
|vl_min_limite                    |0            

In [ ]:

valores_nulos = df_dm.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_dm.columns
])

# Coletar os resultados como um dicionário
nulos_dict = valores_nulos.collect()[0].asDict()

# Converter o dicionário em uma lista de Rows
nulos_lista = [Row(Coluna=k, Valores_Nulos=v) for k, v in nulos_dict.items()]

# Criar um DataFrame Spark a partir da lista
valores_nulos_df = spark.createDataFrame(nulos_lista)

# Exibir os valores nulos por coluna
valores_nulos_df.show(200)


+--------------------+-------------+
|              Coluna|Valores_Nulos|
+--------------------+-------------+
|         ch_cpf_hash|           34|
|       dt_min_adesao|      3102173|
|       dt_max_adesao|      3102173|
|       vl_qnt_cartao|            0|
|vl_min_score_coll...|     67261947|
|vl_max_score_coll...|     67261947|
|  vl_min_dias_atraso|      1579553|
|  vl_max_dias_atraso|      1579553|
|vl_min_saldo_devedor|      2047754|
|vl_max_saldo_devedor|      2047754|
|vl_total_saldo_de...|      2047754|
|       vl_min_limite|            0|
|       vl_max_limite|            0|
|     vl_total_limite|            0|
|        ds_behaviour|     61241937|
|        fl_cpf_ativo|      3102173|
|    dt_ultima_compra|     33057014|
|       fl_tem_dm_app|       428311|
|   dt_cadastro_dmapp|    149325494|
|dt_data_ultimo_ac...|    155272239|
|      fl_conta_avita|    149325494|
| dt_ultimo_movimento|    166726313|
|id_tipo_ultimo_mo...|    166726313|
|vl_valor_ultimo_m...|    166298002|
|

In [ ]:
from pyspark.sql.functions import col, sum, when
from pyspark.sql import Row

# # Contar valores nulos em cada coluna
# valores_nulos = df_dm.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_dm.columns])
# valores_nulos.show()
# Contar valores nulos em cada coluna e coletar os resultados


from pyspark.sql.functions import col, sum, when

# Contar o número total de linhas no DataFrame
total_linhas = df_dm.count()

# Contar valores nulos em cada coluna e coletar os resultados
valores_nulos = df_dm.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_dm.columns
])

# Coletar os resultados como um dicionário
nulos_dict = valores_nulos.collect()[0].asDict()

# Converter o dicionário em uma lista de Rows, adicionando o percentual de valores nulos
nulos_lista = [
    Row(
        Coluna=k, 
        Valores_Nulos=v, 
        Porcentagem_Nulos=round((v / total_linhas) * 100, 2)
    ) for k, v in nulos_dict.items()
]

# Criar um DataFrame Spark a partir da lista
valores_nulos_df = spark.createDataFrame(nulos_lista)

# Exibir os valores nulos e percentuais por coluna
valores_nulos_df.show(valores_nulos_df.count(), truncate=False)


+---------------------------------+-------------+-----------------+
|Coluna                           |Valores_Nulos|Porcentagem_Nulos|
+---------------------------------+-------------+-----------------+
|ch_cpf_hash                      |34           |0.0              |
|dt_min_adesao                    |3102173      |1.79             |
|dt_max_adesao                    |3102173      |1.79             |
|vl_qnt_cartao                    |0            |0.0              |
|vl_min_score_colletion           |67261947     |38.88            |
|vl_max_score_colletion           |67261947     |38.88            |
|vl_min_dias_atraso               |1579553      |0.91             |
|vl_max_dias_atraso               |1579553      |0.91             |
|vl_min_saldo_devedor             |2047754      |1.18             |
|vl_max_saldo_devedor             |2047754      |1.18             |
|vl_total_saldo_devedor           |2047754      |1.18             |
|vl_min_limite                    |0            

In [16]:
# Supondo que você tenha uma lista de colunas categóricas
colunas_categoricas = [coluna for coluna, tipo in df_dm.dtypes if tipo == 'string']

# Contar valores únicos em colunas categóricas
for coluna in colunas_categoricas:
    df_dm.groupBy(coluna).count().orderBy('count', ascending=False).show(5, False)


+----------------------------------------------------------------+-----+
|ch_cpf_hash                                                     |count|
+----------------------------------------------------------------+-----+
|7bbaf3d980a8c3a06bb44fe0c3ca3d87efcae20b7735f22693f67957b09dfd1e|34   |
|41a4f01122b0097c93406bea3a6d8cc0fd137c73df7c2cb68d4156e36d903d85|34   |
|061c2dee0ad60ec17884806c0871947ee023847bd51fb12642ff35ca7d3c5805|34   |
|9df11f9168a7e4bc3cab021a097f05ef910083dff2e556d21fc22056af04ee72|34   |
|638271addead729fe6c70a4d252db1dfddbcf683f4a79bde0d4093a39a97a5d2|34   |
+----------------------------------------------------------------+-----+
only showing top 5 rows



+------------+--------+
|ds_behaviour|count   |
+------------+--------+
|ALTO        |77553721|
|NULL        |61241937|
|BAIXO       |18684089|
|MODERADO    |15535675|
+------------+--------+



+------------+---------+
|fl_cpf_ativo|count    |
+------------+---------+
|N           |137246864|
|S           |32666385 |
|NULL        |3102173  |
+------------+---------+



+-------------+---------+
|fl_tem_dm_app|count    |
+-------------+---------+
|N            |148897183|
|S            |23689928 |
|NULL         |428311   |
+-------------+---------+



+--------------+---------+
|fl_conta_avita|count    |
+--------------+---------+
|NULL          |149325494|
|N             |19387113 |
|S             |4302815  |
+--------------+---------+



...

+-----------------+---------+
|fl_Farmacia_Ativo|count    |
+-----------------+---------+
|N                |167569112|
|NULL             |3102173  |
|S                |2344137  |
+-----------------+---------+



+----------------+---------+
|fl_Mercado_Ativo|count    |
+----------------+---------+
|N               |143167255|
|S               |26745994 |
|NULL            |3102173  |
+----------------+---------+



+-------------------+---------+
|fl_Construcao_Ativo|count    |
+-------------------+---------+
|N                  |169433665|
|NULL               |3102173  |
|S                  |479584   |
+-------------------+---------+



+-------------------+---------+
|fl_Bandeirado_Ativo|count    |
+-------------------+---------+
|N                  |166200013|
|S                  |3713236  |
|NULL               |3102173  |
+-------------------+---------+



+---------------+---------+
|fl_Outros_Ativo|count    |
+---------------+---------+
|N              |168983770|
|NULL           |3102173  |
|S              |929479   |
+---------------+---------+



In [20]:
from pyspark.sql.functions import col, isnan, countDistinct, count

def pod_academy_generate_metadata(spark_df):
    """
    Generates a DataFrame containing metadata of the provided PySpark DataFrame.

    :param spark_df: PySpark DataFrame for which metadata will be generated.
    :return: PySpark DataFrame containing metadata.
    """
    total_rows = spark_df.count()
    
    # List to hold metadata information
    metadata_list = []
    
    for column in spark_df.columns:
        dtype = str(spark_df.schema[column].dataType)
        
        # Count nulls and NaNs
        if dtype in ["DoubleType", "FloatType"]:
            null_count = spark_df.filter(col(column).isNull() | isnan(col(column))).count()
        else:
            null_count = spark_df.filter(col(column).isNull()).count()
        
        percent_nulls = round((null_count / total_rows) * 100, 2)
        
        # Cardinality
        cardinality = spark_df.select(countDistinct(col(column))).collect()[0][0]
        
        metadata_list.append((column, dtype, null_count, percent_nulls, cardinality))
    
    # Create a DataFrame from the metadata list
    metadata_df = spark.createDataFrame(metadata_list, ['nome_variavel', 'tipo', 'qt_nulos', 'percent_nulos', 'cardinalidade'])
    
    # Sort and reset index
    metadata_df = metadata_df.orderBy(col('percent_nulos').desc())
    
    return metadata_df

metadata_df = pod_academy_generate_metadata(df_dm)
metadata_df.show(truncate=False)

+-----------------------------+-----------------+---------+-------------+-------------+
|nome_variavel                |tipo             |qt_nulos |percent_nulos|cardinalidade|
+-----------------------------+-----------------+---------+-------------+-------------+
|dt_data_efetivacao           |DateType()       |172950488|99.96        |744          |
|dt_ultima_compra_dmpag       |DateType()       |172937843|99.96        |673          |
|vl_total_compra              |DecimalType(15,2)|172937843|99.96        |3564         |
|dt_data_contratacao          |DateType()       |172912014|99.94        |791          |
|dt_cadastro_dmpag            |DateType()       |172538882|99.72        |517          |
|fl_dmpag_ativo               |StringType()     |172538882|99.72        |2            |
|vl_qnt_compras_dmpag         |IntegerType()    |172538882|99.72        |20           |
|vl_dias_atraso_dmpag         |IntegerType()    |172538882|99.72        |536          |
|vl_saldo_devedor_dmpag       |D

In [21]:
metadata_df.show(200)

+--------------------+-----------------+---------+-------------+-------------+
|       nome_variavel|             tipo| qt_nulos|percent_nulos|cardinalidade|
+--------------------+-----------------+---------+-------------+-------------+
|  dt_data_efetivacao|       DateType()|172950488|        99.96|          744|
|dt_ultima_compra_...|       DateType()|172937843|        99.96|          673|
|     vl_total_compra|DecimalType(15,2)|172937843|        99.96|         3564|
| dt_data_contratacao|       DateType()|172912014|        99.94|          791|
|   dt_cadastro_dmpag|       DateType()|172538882|        99.72|          517|
|      fl_dmpag_ativo|     StringType()|172538882|        99.72|            2|
|vl_qnt_compras_dmpag|    IntegerType()|172538882|        99.72|           20|
|vl_dias_atraso_dmpag|    IntegerType()|172538882|        99.72|          536|
|vl_saldo_devedor_...|DecimalType(15,2)|172538882|        99.72|         1272|
|vl_dias_atraso_dm...|    IntegerType()|172490259|  

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# Selecionar colunas numéricas
colunas_numericas = [coluna for coluna, tipo in df_dm.dtypes if tipo in ('int', 'double', 'float', 'long')]

# Montar o vetor de recursos
assembler = VectorAssembler(inputCols=colunas_numericas, outputCol="features")
df_vector = assembler.transform(df_dm.select(colunas_numericas)).select("features")

# Calcular a matriz de correlação
matriz_correlacao = Correlation.corr(df_vector, "features").collect()[0][0]
print("Matriz de correlação:\n" + str(matriz_correlacao))


In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histograma de uma coluna numérica
coluna_numerica = colunas_numericas[0]  # Por exemplo, a primeira coluna numérica
sns.histplot(amostra_df[coluna_numerica], kde=True)
plt.title(f'Histograma de {coluna_numerica}')
plt.show()

# Gráfico de barras para uma coluna categórica
coluna_categorica = colunas_categoricas[0]  # Por exemplo, a primeira coluna categórica
sns.countplot(y=coluna_categorica, data=amostra_df)
plt.title(f'Contagem de {coluna_categorica}')
plt.show()


Matplotlib is building the font cache; this may take a moment.


NameError: name 'colunas_numericas' is not defined

In [ ]:
# Amostrar 1% dos dados
amostra_df = df_dm.sample(fraction=0.01, seed=42).toPandas()


In [ ]:
pandas_df = df_dm.toPandas().convert_dtypes()
pandas_df['dt_max_adesao'] = pd.to_datetime(pandas_df['dt_max_adesao'])
pandas_df['dt_min_adesao'] = pd.to_datetime(pandas_df['dt_min_adesao'])
pandas_df['dt_fechamento'] = pd.to_datetime(pandas_df['dt_fechamento'])
pandas_df['vl_min_limite'] = pd.to_numeric(pandas_df['vl_min_limite'].astype(float))
pandas_df['vl_max_limite'] = pd.to_numeric(pandas_df['vl_max_limite'].astype(float))
pandas_df['vl_total_limite'] = pd.to_numeric(pandas_df['vl_total_limite'].astype(float))
pandas_df['vl_min_saldo_devedor'] = pd.to_numeric(pandas_df['vl_min_saldo_devedor'].astype(float))
pandas_df['vl_max_saldo_devedor'] = pd.to_numeric(pandas_df['vl_max_saldo_devedor'].astype(float))
pandas_df['vl_total_saldo_devedor'] = pd.to_numeric(pandas_df['vl_total_saldo_devedor'].astype(float))

In [ ]:
# 1. Unpivot the product-specific columns into a long format DataFrame
product_categories = ['Farmacia', 'Mercado', 'Construcao', 'Bandeirado', 'Emprestimo', 'Outros']
product_columns = [col for col in df_dm.columns if any(col.startswith(f'vl_{cat}_') for cat in product_categories)]

# Create a list of tuples (product_category, column_name)
unpivot_exprs = [(cat, col) for cat in product_categories for col in product_columns if col.startswith(f'vl_{cat}_')]

# Convert all product-specific columns to string
for cat, col in unpivot_exprs:
    df_dm = df_dm.withColumn(col, F.col(col).cast("string"))

# Unpivot the DataFrame
unpivoted_df = df_dm.select(
    'ch_cpf_hash', 'dt_min_adesao', 'dt_max_adesao', 'vl_qnt_cartao', 'vl_min_score_colletion', 'vl_max_score_colletion',
    'vl_min_dias_atraso', 'vl_max_dias_atraso', 'vl_min_saldo_devedor', 'vl_max_saldo_devedor', 'vl_total_saldo_devedor',
    'vl_min_limite', 'vl_max_limite', 'vl_total_limite', 'ds_behaviour', 'fl_cpf_ativo', 'dt_ultima_compra', 'fl_tem_dm_app',
    'dt_cadastro_dmapp', 'dt_data_ultimo_acesso_app', 'fl_conta_avita', 'dt_ultimo_movimento', 'id_tipo_ultimo_movimento',
    'vl_valor_ultimo_movimento', 'ds_categoria_ultimo_movimento', 'dt_data_saldo', 'vl_saldo_dm_conta', 'fl_tem_dmcred',
    'id_cliente_nivel', 'dt_data_contratacao', 'dt_data_efetivacao', 'id_ciclo', 'vl_dias_atraso_dmcred', 'vl_saldo_devedor_dmcred',
    'fl_tem_dmpag', 'dt_cadastro_dmpag', 'fl_dmpag_ativo', 'dt_ultima_compra_dmpag', 'vl_qnt_compras_dmpag', 'vl_total_compra',
    'vl_dias_atraso_dmpag', 'vl_saldo_devedor_dmpag', 'dt_fechamento',
    F.expr(f"stack({len(unpivot_exprs)}, " + ", ".join([f"'{cat}', {col}" for cat, col in unpivot_exprs]) + ") as (product_name, value)")
)

# 2. Create a new column 'product_name' that indicates the product category
# This step is already handled in the unpivoting process

# 3. Consolidate product-specific measures under common column names
# Assuming 'value' column contains the consolidated measures

# 4. Prepare the DataFrame for creating the 'fact_product_snapshot' fact table
fact_product_snapshot = unpivoted_df.withColumnRenamed('value', 'measure_value')

# 5. Create dimension tables
# dim_customer
dim_customer = df_dm.select(
    'ch_cpf_hash', 'ds_behaviour', 'fl_cpf_ativo', 'fl_tem_dm_app', 'fl_conta_avita', 'fl_tem_dmcred', 'fl_tem_dmpag'
).distinct()

# dim_product
dim_product = spark.createDataFrame(
    [(i, cat) for i, cat in enumerate(product_categories, start=1)],
    ['product_id', 'product_name']
)

# dim_date
date_columns = ['dt_min_adesao', 'dt_max_adesao', 'dt_ultima_compra', 'dt_cadastro_dmapp', 'dt_data_ultimo_acesso_app',
                'dt_ultimo_movimento', 'dt_data_saldo', 'dt_data_contratacao', 'dt_data_efetivacao', 'dt_cadastro_dmpag',
                'dt_ultima_compra_dmpag', 'dt_fechamento']

# Rename date columns to avoid ambiguity
dim_date = df_dm.select([F.col(col).alias(f'{col}_date') for col in date_columns]).distinct()

# Add surrogate key to dim_date
dim_date = dim_date.withColumn('date_id', F.row_number().over(Window.orderBy(F.col('dt_min_adesao_date'))))

# Show the resulting DataFrames
fact_product_snapshot.show()
dim_customer.show()
dim_product.show()
dim_date.show()

24/11/23 16:47:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-------------+-------------+-------------+----------------------+----------------------+------------------+------------------+--------------------+--------------------+----------------------+-------------+-------------+---------------+------------+------------+----------------+-------------+-----------------+-------------------------+--------------+-------------------+------------------------+-------------------------+-----------------------------+-------------+-----------------+-------------+----------------+-------------------+------------------+--------+---------------------+-----------------------+------------+-----------------+--------------+----------------------+--------------------+---------------+--------------------+----------------------+-------------+------------+-------------+
|         ch_cpf_hash|dt_min_adesao|dt_max_adesao|vl_qnt_cartao|vl_min_score_colletion|vl_max_score_colletion|vl_min_dias_atraso|vl_max_dias_atraso|vl_min_saldo_devedor|vl_max_s

24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/23 16:47:57 WARN RowBasedKeyValueBatch: Calling spill() on

In [ ]:
# Verificar os tipos de dados de cada coluna
dtypes = df.dtypes

# Mostrar os tipos de dados
for column, dtype in dtypes:
    print(f"Coluna: {column}, Tipo de dado: {dtype}")

# Selecionar colunas qualitativas
qualitative_columns = ["ch_cpf_hash", "dt_fechamento"]
df_qualitative = df.select(*qualitative_columns)

# Mostrar as primeiras linhas do DataFrame qualitativo
df_qualitative.show(5)

+--------------------+-------------+
|         ch_cpf_hash|dt_fechamento|
+--------------------+-------------+
|0da564d51ee1ce79b...|   2024-08-31|
|d2a444414ab222bf5...|   2024-08-31|
|39e26353f2a9de3b0...|   2024-08-31|
|b87362d224305c9c0...|   2024-08-31|
|f9978d9d96ae87331...|   2024-08-31|
+--------------------+-------------+
only showing top 5 rows



In [18]:
# Selecionar colunas quantitativas
quantitative_columns = ["vl_qnt_conta", "vl_min_limite_aprovado", "vl_max_limite_aprovado", "vl_total_limite_aprovado", "min_saldo_devedor", "max_saldo_devedor", "total_saldo_devedor", "min_dias_atraso", "max_dias_atraso"]
df_quantitative = df.select(*quantitative_columns)

# Mostrar as primeiras linhas do DataFrame quantitativo
#df_quantitative.show(5)
# df_quantitative.describe().show() 

In [ ]:
# Converter para Pandas DataFrame para melhor visualização
df_describe_pandas = df_quantitative.describe().toPandas().convert_dtypes()

# Mostrar o DataFrame descritivo
df_describe_pandas

,summary,vl_qnt_conta,vl_min_limite_aprovado,vl_max_limite_aprovado,vl_total_limite_aprovado,min_saldo_devedor,max_saldo_devedor,total_saldo_devedor,min_dias_atraso,max_dias_atraso
0,count,1127650,1127640,1127640,1127640,1127650,1127650,1127650,1127650,1127650
1,mean,1.000527645989447,2124.722917,2125.125089,2126.672298,1270.635294,1271.083477,1271.418375,104.92518777989625,105.02991353700173
2,stddev,0.11290475029520305,1622.0309105320182,1622.3318266473307,1636.7232121960942,2404.524805272714,2404.7404514493346,2405.441601953322,232.53130138880078,232.8985853235725
3,min,1,0.00,0.00,0.00,-12464.40,-12464.40,-12464.40,0,0
4,max,54,12500.00,12500.00,96700.00,56578.72,56578.72,56578.72,2762,2762


In [30]:
df

DataFrame[ch_cpf_hash: string, vl_qnt_conta: bigint, dt_min_adesao: date, dt_max_adesao: date, vl_min_limite_aprovado: decimal(7,2), vl_max_limite_aprovado: decimal(7,2), vl_total_limite_aprovado: decimal(38,2), min_saldo_devedor: decimal(38,2), max_saldo_devedor: decimal(38,2), total_saldo_devedor: decimal(38,2), min_dias_atraso: int, max_dias_atraso: int, dt_fechamento: date]

In [31]:
import pandas_profiling as pp
import pandas as pd


# Converter para pandas DataFrame
pandas_df = df.toPandas().convert_dtypes()
pandas_df['dt_max_adesao'] = pd.to_datetime(pandas_df['dt_max_adesao'])
pandas_df['dt_min_adesao'] = pd.to_datetime(pandas_df['dt_min_adesao'])
pandas_df['dt_fechamento'] = pd.to_datetime(pandas_df['dt_fechamento'])
pandas_df['vl_min_limite_aprovado'] = pd.to_numeric(pandas_df['vl_min_limite_aprovado'].astype(float))
pandas_df['vl_max_limite_aprovado'] = pd.to_numeric(pandas_df['vl_max_limite_aprovado'].astype(float))
pandas_df['vl_total_limite_aprovado'] = pd.to_numeric(pandas_df['vl_total_limite_aprovado'].astype(float))
pandas_df['min_saldo_devedor'] = pd.to_numeric(pandas_df['min_saldo_devedor'].astype(float))
pandas_df['max_saldo_devedor'] = pd.to_numeric(pandas_df['max_saldo_devedor'].astype(float))
pandas_df['total_saldo_devedor'] = pd.to_numeric(pandas_df['total_saldo_devedor'].astype(float))

# Gerar relatório com pandas-profiling
profile = pp.ProfileReport(pandas_df)
profile.to_file("relatorio.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
pandas_df.head(2)

,ch_cpf_hash,vl_qnt_conta,dt_min_adesao,dt_max_adesao,vl_min_limite_aprovado,vl_max_limite_aprovado,vl_total_limite_aprovado,min_saldo_devedor,max_saldo_devedor,total_saldo_devedor,min_dias_atraso,max_dias_atraso,dt_fechamento
0,0da564d51ee1ce79bd60bd0b723238c3878aa1d54d6da4...,1,2021-12-27,2021-12-27,600.00,600.00,600.00,307.78,307.78,307.78,149,149,2024-08-31
1,d2a444414ab222bf58b39090d7e64a66b2c1cdef2dad37...,1,2021-12-28,2021-12-28,1900.00,1900.00,1900.00,2592.25,2592.25,2592.25,220,220,2024-08-31


In [29]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Resumindo a tabela
def summarize_table(df):
    print("Resumindo a tabela...")

    # Número de linhas e colunas
    num_rows = df.count()
    num_cols = len(df.columns)
    print(f" Número de Linhas: {num_rows}")
    print(f" Número de Colunas: {num_cols}")

    # Tipos de dados das colunas
    print("\n Tipos de Dados das Colunas:")
    df.printSchema()

    # Estatísticas descritivas
    print("\n Estatísticas Descritivas:")
    df.describe().show()

# Chamando a função
summarize_table(df)

Resumindo a tabela...
 Número de Linhas: 1127650
 Número de Colunas: 13

 Tipos de Dados das Colunas:
root
 |-- ch_cpf_hash: string (nullable = true)
 |-- vl_qnt_conta: long (nullable = true)
 |-- dt_min_adesao: date (nullable = true)
 |-- dt_max_adesao: date (nullable = true)
 |-- vl_min_limite_aprovado: decimal(7,2) (nullable = true)
 |-- vl_max_limite_aprovado: decimal(7,2) (nullable = true)
 |-- vl_total_limite_aprovado: decimal(38,2) (nullable = true)
 |-- min_saldo_devedor: decimal(38,2) (nullable = true)
 |-- max_saldo_devedor: decimal(38,2) (nullable = true)
 |-- total_saldo_devedor: decimal(38,2) (nullable = true)
 |-- min_dias_atraso: integer (nullable = true)
 |-- max_dias_atraso: integer (nullable = true)
 |-- dt_fechamento: date (nullable = true)


 Estatísticas Descritivas:


+-------+--------------------+-------------------+----------------------+----------------------+------------------------+-----------------+------------------+-------------------+------------------+------------------+
|summary|         ch_cpf_hash|       vl_qnt_conta|vl_min_limite_aprovado|vl_max_limite_aprovado|vl_total_limite_aprovado|min_saldo_devedor| max_saldo_devedor|total_saldo_devedor|   min_dias_atraso|   max_dias_atraso|
+-------+--------------------+-------------------+----------------------+----------------------+------------------------+-----------------+------------------+-------------------+------------------+------------------+
|  count|             1127650|            1127650|               1127640|               1127640|                 1127640|          1127650|           1127650|            1127650|           1127650|           1127650|
|   mean|                NULL|  1.000527645989447|           2124.722917|           2125.125089|             2126.672298|      1270.

In [22]:
pandas_df.dtypes

ch_cpf_hash                 string
vl_qnt_conta                 Int64
dt_min_adesao               object
dt_max_adesao               object
vl_min_limite_aprovado      object
vl_max_limite_aprovado      object
vl_total_limite_aprovado    object
min_saldo_devedor           object
max_saldo_devedor           object
total_saldo_devedor         object
min_dias_atraso              Int32
max_dias_atraso              Int32
dt_fechamento               object
dtype: object

# Explorando o DataFrame
.

In [7]:
# Obter a contagem de valores únicos na coluna 'vl_qnt_conta'
value_counts_cpf = df.groupBy("ch_cpf_hash").count().orderBy("count", ascending=False)
value_counts_qnt_con = df.groupBy("vl_qnt_conta").count().orderBy("count", ascending=False)
value_counts_min_ads = df.groupBy("dt_min_adesao").count().orderBy("count", ascending=False)
value_counts_max_ads = df.groupBy("dt_max_adesao").count().orderBy("count", ascending=False)
val_cnt_min_lmt_apr = df.groupBy("vl_min_limite_aprovado").count().orderBy("count", ascending=False)
vl_max_limite_aprovado = df.groupBy("vl_max_limite_aprovado").count().orderBy("count", ascending=False)
vl_total_limite_aprovado = df.groupBy("vl_total_limite_aprovado").count().orderBy("count", ascending=False)
min_saldo_devedor = df.groupBy("min_saldo_devedor").count().orderBy("count", ascending=False)
max_saldo_devedor = df.groupBy("max_saldo_devedor").count().orderBy("count", ascending=False)
total_saldo_devedor = df.groupBy("total_saldo_devedor").count().orderBy("count", ascending=False)
min_dias_atraso = df.groupBy("min_dias_atraso").count().orderBy("count", ascending=False)
max_dias_atraso = df.groupBy("max_dias_atraso").count().orderBy("count", ascending=False)
dt_fechamento = df.groupBy("dt_fechamento").count().orderBy("count", ascending=False)

# Mostrar o resultado
value_counts_cpf.show()
value_counts_qnt_con.show()
value_counts_min_ads.show()
value_counts_max_ads.show()
val_cnt_min_lmt_apr.show()
vl_max_limite_aprovado.show()
vl_total_limite_aprovado.show()
min_saldo_devedor.show()
max_saldo_devedor.show()
total_saldo_devedor.show()
min_dias_atraso.show()
max_dias_atraso.show()
dt_fechamento.show()

+--------------------+-----+
|         ch_cpf_hash|count|
+--------------------+-----+
|038ffd39fb58c5f1b...|    5|
|3d79d5e702a97715c...|    5|
|0c0166d66319820fc...|    5|
|9806dde903c8a1bfa...|    5|
|86954a39d2a775d17...|    5|
|fd9761d637c3ce969...|    5|
|98199d258b3cbf426...|    5|
|1a08db347c20a4384...|    5|
|4286661ba0a8c231e...|    5|
|8bbeb0febfd217152...|    5|
|ebd93149f5719b433...|    5|
|23fbbd327853c8039...|    5|
|69a88a3fc9f492197...|    5|
|9ecee4185f4bdf0e4...|    5|
|7e676758c856f3512...|    5|
|264a1aa26619c3782...|    5|
|a206f6d3f06094a98...|    5|
|7d368838ce524dc4b...|    5|
|bce661a71eaf5bec7...|    5|
|aecb26c555bdea77d...|    5|
+--------------------+-----+
only showing top 20 rows

+------------+-------+
|vl_qnt_conta|  count|
+------------+-------+
|           1|1127315|
|           2|    330|
|          54|      5|
+------------+-------+

+-------------+-----+
|dt_min_adesao|count|
+-------------+-----+
|   2022-08-17| 3740|
|   2021-01-04| 3675|
|   20

+-----------------+------+
|min_saldo_devedor| count|
+-----------------+------+
|             0.00|167362|
|            13.90|  8993|
|             0.02|  7502|
|             0.04|  5345|
|             0.06|  5061|
|             0.03|  4402|
|             0.01|  4119|
|             0.05|  3882|
|             0.07|  2942|
|            -0.01|  2899|
|             0.08|  2708|
|             0.10|  2166|
|             0.09|  2120|
|            -0.50|  2007|
|            -1.00|  1989|
|            23.89|  1808|
|             0.11|  1519|
|             0.12|  1416|
|             0.14|  1409|
|             0.66|  1339|
+-----------------+------+
only showing top 20 rows

+-----------------+------+
|max_saldo_devedor| count|
+-----------------+------+
|             0.00|167331|
|            13.90|  8992|
|             0.02|  7492|
|             0.04|  5350|
|             0.06|  5061|
|             0.03|  4402|
|             0.01|  4124|
|             0.05|  3887|
|             0.07|  2947|
| 

In [9]:
# Mostrar as primeiras linhas do DataFrame
df.show(5)

# Mostrar o número de linhas e colunas do DataFrame
num_rows = df.count()
num_cols = len(df.columns)
print(f"O DataFrame concatenado tem {num_rows} linhas e {num_cols} colunas.")

+--------------------+------------+-------------+-------------+----------------------+----------------------+------------------------+-----------------+-----------------+-------------------+---------------+---------------+-------------+
|         ch_cpf_hash|vl_qnt_conta|dt_min_adesao|dt_max_adesao|vl_min_limite_aprovado|vl_max_limite_aprovado|vl_total_limite_aprovado|min_saldo_devedor|max_saldo_devedor|total_saldo_devedor|min_dias_atraso|max_dias_atraso|dt_fechamento|
+--------------------+------------+-------------+-------------+----------------------+----------------------+------------------------+-----------------+-----------------+-------------------+---------------+---------------+-------------+
|0da564d51ee1ce79b...|           1|   2021-12-27|   2021-12-27|                600.00|                600.00|                  600.00|           307.78|           307.78|             307.78|            149|            149|   2024-08-31|
|d2a444414ab222bf5...|           1|   2021-12-28|   

# Transformações e Ações em DataFrame Spark
Execute transformações e ações em DataFrames Spark para manipular e analisar os dados.

In [10]:
# Remover linhas com valores nulos em colunas importantes
df_clean = df.dropna(subset=["ch_cpf_hash", "dt_fechamento", "vl_qnt_conta"])

# Verificar novamente o número de linhas após a limpeza
num_rows_clean = df_clean.count()
print(f"O DataFrame limpo tem {num_rows_clean} linhas.")

O DataFrame limpo tem 1127650 linhas.


# Salvar Resultados
Salve os resultados das operações em um formato de arquivo ou banco de dados.

In [12]:
# Supondo que existam colunas como 'idade' e 'sexo' no DataFrame
# Calcular a distribuição de idade
df_clean.groupBy("vl_min_limite_aprovado").count().orderBy("vl_min_limite_aprovado").show()

# Calcular a distribuição de sexo
df_clean.groupBy("vl_qnt_conta").count().show()

+----------------------+-----+
|vl_min_limite_aprovado|count|
+----------------------+-----+
|                  NULL|   10|
|                  0.00|   45|
|                100.00|   15|
|                300.00|    5|
|                500.00|39035|
|                600.00|85125|
|                700.00|29610|
|                727.21|    5|
|                800.00|25515|
|                841.42|    5|
|                900.00|31705|
|                915.53|    5|
|                930.88|    5|
|                944.51|    5|
|                945.58|    5|
|                950.00|    5|
|                972.00|    5|
|               1000.00|44965|
|               1035.93|    5|
|               1040.48|    5|
+----------------------+-----+
only showing top 20 rows

+------------+-------+
|vl_qnt_conta|  count|
+------------+-------+
|          54|      5|
|           1|1127315|
|           2|    330|
+------------+-------+



In [13]:
# Criar segmentos com base em critérios específicos, por exemplo, idade e saldo devedor
df_segmented = df_clean.withColumn(
    "segmento",
    when((col("idade") < 30) & (col("total_saldo_devedor") < 1000), "Jovens com baixo saldo devedor")
    .when((col("idade") >= 30) & (col("total_saldo_devedor") >= 1000), "Adultos com alto saldo devedor")
    .otherwise("Outros")
)

# Mostrar a distribuição dos segmentos
df_segmented.groupBy("segmento").count().show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `idade` cannot be resolved. Did you mean one of the following? [`dt_max_adesao`, `dt_min_adesao`, `ch_cpf_hash`, `dt_fechamento`, `min_dias_atraso`].;
'Project [ch_cpf_hash#135, vl_qnt_conta#136L, dt_min_adesao#137, dt_max_adesao#138, vl_min_limite_aprovado#139, vl_max_limite_aprovado#140, vl_total_limite_aprovado#141, min_saldo_devedor#142, max_saldo_devedor#143, total_saldo_devedor#144, min_dias_atraso#145, max_dias_atraso#146, dt_fechamento#147, CASE WHEN (('idade < 30) AND (total_saldo_devedor#144 < cast(cast(1000 as decimal(4,0)) as decimal(38,2)))) THEN Jovens com baixo saldo devedor WHEN (('idade >= 30) AND (total_saldo_devedor#144 >= cast(cast(1000 as decimal(4,0)) as decimal(38,2)))) THEN Adultos com alto saldo devedor ELSE Outros END AS segmento#335]
+- Filter atleastnnonnulls(3, ch_cpf_hash#135, dt_fechamento#147, vl_qnt_conta#136L)
   +- Relation [ch_cpf_hash#135,vl_qnt_conta#136